In [ ]:
import praw
import time
from datetime import datetime
import json
from json import load, dump


In [ ]:
# Load Reddit credentials from environment (.env) to avoid committing secrets
from dotenv import load_dotenv
import os
load_dotenv()
REDDIT_CLIENT_ID = os.environ.get("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.environ.get("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT = os.environ.get("REDDIT_USER_AGENT", "social-media-a2/0.1 by unknown")
if not REDDIT_CLIENT_ID or not REDDIT_CLIENT_SECRET:
    raise RuntimeError("REDDIT_CLIENT_ID and REDDIT_CLIENT_SECRET are required. Add them to .env or export in your environment.")

reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT,
)

combined_data = []

# subreddit_names = ["UkraineRussiaReport", "RussiaUkraineWar", "RussiaUkraineWar2022", "UkrainianConflict"]
subreddit_names = ["RussiaUkraineWar2022"]
print("📥 Fetching posts and comments...")

for subreddit_name in subreddit_names:
    subreddit = reddit.subreddit(subreddit_name)

    for idx, submission in enumerate(subreddit.hot(limit=1000)):
        print(f"🔎 Processing post {idx + 1}: {submission.title}")

        # Add submission (post) details
        combined_data.append({
            'id': submission.id,
            'type': 'submission',
            'subreddit': submission.subreddit.display_name,
            'title': submission.title,
            'selftext': submission.selftext,
            'created_utc': submission.created_utc,
            'created': datetime.utcfromtimestamp(submission.created_utc).isoformat(),
            'author': str(submission.author),
            'score': submission.score,
            'upvote_ratio': submission.upvote_ratio,
            'num_comments': submission.num_comments,
            'url': submission.url,
            'permalink': f"https://reddit.com{submission.permalink}"
        })

        # Fetch top 50 comments
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list()[:50]:
            combined_data.append({
                'id': comment.id,
                'type': 'comment',
                'parent_id': comment.parent_id,
                'link_id': comment.link_id,
                'subreddit': comment.subreddit.display_name,
                'body': comment.body,
                'author': str(comment.author),
                'created_utc': comment.created_utc,
                'created': datetime.utcfromtimestamp(comment.created_utc).isoformat(),
                'score': comment.score,
                'permalink': f"https://reddit.com{comment.permalink}" if hasattr(comment, 'permalink') else None
            })

        if (idx + 1) % 10 == 0:
            print(f"⏳ Processed {idx + 1} posts...")
            time.sleep(1)

# Save to JSON
output_path = "reddit_data_RussiaUkraineWar2022.json"
with open(output_path, "w", encoding='utf-8') as f:
    json.dump(combined_data, f, ensure_ascii=False, indent=4)

print(f"✅ Done! Saved {len(combined_data)} records to {output_path}")

📥 Fetching posts and comments...
🔎 Processing post 1: I want to ask EVERY Russian
🔎 Processing post 2: Vladimir Putin's Russian forces 'poised to launch major new Ukraine offensive'
🔎 Processing post 3: Russian hackers target Western firms shipping aid to Ukraine, US intelligence says
🔎 Processing post 4: Russia has never sought peace - not in Chechnya, not in Syria, not in Africa, not in Crimea, not in Bucha, not now.Every “peace initiative” from Moscow is a regrouping tactic, not a step toward justice. Their idea of peace is silence over the ruins of others. War is Russia's peace!
🔎 Processing post 5: Germany’s defense minister Boris Pistorius says Trump’s talk with Putin shows Moscow won’t compromise. As Russia ignores peace efforts, EU leaders push for new sanctions, while Trump declines further action. Continued military support for Ukraine remains crucial
🔎 Processing post 6: Adam Zivo: Putin outplays Trump again with phoney peace talks
🔎 Processing post 7: Just one AFU drone sca

In [ ]:
with open("reddit_data_RussiaUkraineWar2022.json") as j1, open("reddit_data_ukraine_war.json") as j2, open("reddit_data_combined.json", "w") as j3:
    dump([load(j1), load(j2)], j3)

In [ ]:
with open('youtube_war_comments.json', 'r') as f1:
    data1 = json.load(f1)

with open('youtube_vs_comments.json', 'r') as f2:
    data2 = json.load(f2)

In [ ]:
combined_data = {**data1, **data2}

In [19]:
with open('combined_youtube.json', 'w') as outfile:
    json.dump(combined_data, outfile, indent=4)